In [1]:
# Install required libraries
!pip install pytorch torchvision torchaudio
!pip install transformers==4.30
!pip install langchain sentence_transformers huggingface-hub
!pip install pypdf
!pip install -U langchain-community
!pip install bitsandbytes
!pip install faiss-cpu langchain-openai tiktoken unstructured selenium newspaper3k textstat
!pip install accelerate

!pip install langchain-huggingface
!pip install sentence-transformers==2.2.2
!pip install InstructorEmbedding

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pytorch)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached transformers-4.43.3-py3-none-any.whl.metadata (43 kB)
Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
Using cached transformers-4.43.3-py3-none-any.whl (9.

In [2]:
from google.colab import drive, userdata
import os
import pickle
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings


In [3]:
# Mount Google Drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/MyDrive/WAI_project/"

Mounted at /content/gdrive


In [4]:
# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [5]:
# Set HF token
hf_token = userdata.get('HF_TOKEN')

In [6]:
# load instructor embeddings model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": device})

load INSTRUCTOR_Transformer
max_seq_length  512


In [7]:
# set local path to store embeddings ***replace with SingleStore, AWS or similar
embedding_store_path = f"{root_dir}/embedding_store"

In [8]:
# defines the parameters to use the recursive text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 10,
    length_function = len,
)

In [9]:
def ingest_and_chunk_pdfs():
    loader = DirectoryLoader(f"{root_dir}", glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()

    texts = text_splitter.split_documents(documents)

    return texts

In [10]:
# Function to store embeddings
def store_embeddings(docs, embeddings, store_name, path):
    """
    Stores embeddings in FAISS format and saves to a pickle file.

    Args:
    - docs (list): List of documents.
    - embeddings: Embedding model.
    - store_name (str): Name of the embedding store.
    - path (str): Path to the directory where embeddings will be stored.
    """
    vector_store = FAISS.from_documents(docs, embeddings)
    with open(os.path.join(path, f"faiss_{store_name}.pkl"), "wb") as f:
        pickle.dump(vector_store, f)

In [11]:
# Function to load embeddings
def load_embeddings(store_name, path):
    """
    Loads embeddings from a pickle file.

    Args:
    - store_name (str): Name of the embedding store.
    - path (str): Path to the directory where embeddings are stored.

    Returns:
    - vector_store: Loaded FAISS vector store.
    """
    with open(os.path.join(path, f"faiss_{store_name}.pkl"), "rb") as f:
        vector_store = pickle.load(f)
    return vector_store

In [12]:
# Function to initialize Hugging Face Instruct Embeddings
def initialize_huggingface_embeddings(model_name="hkunlp/instructor-xl", device="cuda"):
    """
    Initializes Hugging Face Instruct Embeddings model.

    Args:
    - model_name (str): Name of the Hugging Face model.
    - device (str): Device to run the model on.

    Returns:
    - embeddings: Initialized Hugging Face Instruct Embeddings model.
    """
    return HuggingFaceInstructEmbeddings(model_name=model_name, model_kwargs={"device": device})


In [13]:
def retrieve_relevant_chunks(question, vector_store, num_chunks=1):
    docs = vector_store.similarity_search(question, k=num_chunks)
    return docs

In [14]:
def format_prompt(question, chunks):
    context = "\n".join([chunk.page_content for chunk in chunks])
    prompt = f"Provide an answer to the following question using only the context provided: {question}? " \
             f"If you cannot answer this question from the information provided, respond with 'There is insufficient information to answer this question.'\n\n{context}"
    return prompt

def initialize_model_and_tokenizer():
    quantization_config = BitsAndBytesConfig(load_in_4bit=True)
    model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", quantization_config=quantization_config)
    tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b", quantization_config=quantization_config, model_max_length=256)
    return model, tokenizer

def gen_answer(prompt, tokenizer, model, max_length=100, temperature=0.7):
    inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        response = model.generate(inputs, max_new_tokens=max_length, temperature=temperature)
    answer = tokenizer.decode(response[0], skip_special_tokens=True)
    return answer.strip()

In [15]:
def main(question):
    torch.cuda.empty_cache()

    # Initialize embedding model
    instructor_embeddings = initialize_huggingface_embeddings()

    # Read and chunk data
    texts = ingest_and_chunk_pdfs()

    # Create and store embeddings
    store_embeddings(texts, instructor_embeddings, store_name='instructEmbeddings', path=embedding_store_path)

    # Load the vector store
    vector_store = load_embeddings(store_name='instructEmbeddings', path=embedding_store_path)

    # Load model and tokenizer with quantization
    model, tokenizer = initialize_model_and_tokenizer()

    # Retrieve relevant chunks based on the question
    relevant_chunks = retrieve_relevant_chunks(question, vector_store)

    # Format the prompt for the LLM
    prompt = format_prompt(question, relevant_chunks)

    # Generate the answer using the LLM and passing model and tokenizer
    answer = gen_answer(prompt, tokenizer, model)
    return answer


In [16]:
# Example usage
question = "What are the main causes of climate change?"
answer = main(question)
print(answer)

load INSTRUCTOR_Transformer
max_seq_length  512


`low_cpu_mem_usage` was None, now set to True since model is quantized.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:430: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Provide an answer to the following question using only the context provided: What are the main causes of climate change?? If you cannot answer this question from the information provided, respond with 'There is insufficient information to answer this question.'

Causes  
The natural causes of climate change should  be examined at different timescales , 
together with consequent climatological events:  
 Geological -scale external forcing mechanisms , such as  the three types of 
Milankovitch cycle (eccentricity, tilt and precession) , are responsible for defining 
the scope of the Quaternary  period . 
 Within the Quaternary, the Pleistocene and more recently the Holocene  periods , 
the Milankovitch cycles are superimposed on the tectonic and isostatic cycles. 
 The tectonic and isostatic cycles are superimposed on the Milankovitch cycles. 
 The Milankovitch cycles are superimposed on the Milankovitch cycles. 
 The Milankovitch cycles are superimposed on the Milankovitch cycles. 